# Portfolio Price Analytics

This iPython notebook serves to provide simple but important stock price insights using stock portfolio history data from the excel file located in the same directory. It imports real-time stock data using the yahoo-finance python module and is uses simple difference analysis to see which shares are doing well. 
https://pypi.python.org/pypi/yahoo-finance  

In [ ]:
import pandas as pd
import yahoo_finance # https://pypi.python.org/pypi/yahoo-finance, https://github.com/lukaszbanasiak/yahoo-finance
import matplotlib.pyplot as plt 
%matplotlib inline
%pylab inline
#pylab.rcParams['figure.figsize'] = (12, 6)   # Change the size of plots
import seaborn as sns
sns.set(color_codes=True)

#cheat sheets 
# http://www.webpages.uidaho.edu/~stevel/504/Pandas%20DataFrame%20Notes.pdf

In [ ]:
#import personal data from portfolio.xlsx
raw_data = pd.read_excel('portfolio.xlsx')
stock_data = raw_data.set_index("stock")
stock_data

In [ ]:
# Get prices from Yahoo-Finance 
shares = [yahoo_finance.Share(str(stock)) for stock in stock_data.index]
current_prices = [share.get_price() for share in shares]
stock_prices = {str(k): float(v) for k, v in zip(stock_data.index, current_prices)}
print (stock_prices)

In [ ]:
df2 = pd.DataFrame({'price_paid' : stock_data['price-paid'], 
                    'current_price': [float(price) for price in current_prices],
                    'quantity' : stock_data['quantity']
                    })

price_differentials = [(x - y) for x, y in zip(df2['current_price'], df2['price_paid'])]
df2.insert(0, 'stock', stock_data.index)
df2.insert(3, 'price_diff', price_differentials)
df2.insert(5, 'values', df2['current_price']*df2['quantity'])
df2.insert(6, 'total_invested', df2['price_paid']*df2['quantity'])
df2.insert(7, 'profit', stock_data['quantity'] * df2['price_diff'])
master_data = df2.set_index('stock')
master_data

In [ ]:
stock = master_data['values'].sum()

def totalAssets(df3, not_invested): 
    return master_data['total_invested'].sum() + not_invested

buying_power = .02
totalAssets = totalAssets(stock_data, buying_power)

df = pd.DataFrame({'total_assets':  [totalAssets], 
                    'stock_valuation':  [stock]})
df

In [ ]:
master_data['profit'].plot(kind='bar', color= 'teal'); plt.axhline(0, color='k')

In [ ]:
profit = 0
positive_profit = 0 
print ('stocks I can sell:')
for stock in master_data.index:
    value = master_data.at[str(stock), 'profit']
    profit += value
    if (value > 0): positive_profit+= value
    if (value > 0): print (stock, 'profit would be:', '$', value)
     
print('positive stock profit:', '$',positive_profit)

print()        
if profit < 0: print ('rip, you have a negative portfolio')
else: print('Dude nice! Positive Portfolio boys!')
print ('expected profit: ', profit)